In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from signals import *
from frequencyestimator import *
import time
import copy

sns.set_style("whitegrid")
sns.despine(left=True, bottom=True)
sns.set_context("poster", font_scale = .45, rc={"grid.linewidth": 0.8})

<Figure size 640x480 with 0 Axes>

In [2]:
from math import factorial

def index_I(q, N, l, i):
    s1 = 0
    s2 = 0
    for j in range(l):
        s1 += N**(q - j - 1) * i[j]
    for j in range(q-l):
        s2 += N**(q - l - j - 1) * i[q + j]
    
    return s1 + s2

def index_J(q, N, l, i):
    s1 = 0
    s2 = 0
    for j in range(l):
        s1 += N**(q - j - 1) * i[2*q - l + j]
    for j in range(q-l):
        s2 += N**(q - l - j - 1) * i[l + j]
    
    return s1 + s2

def comb_factor(q):
    s = 0
    for p in range(1, 2*q + 1):
        s += (-1)**p * factorial(p-1)
    return s

def cumulant_signal(signal, i):
    res = 1
    for n in range(len(i)):
        if n < len(i)//2:
            res = res * signal[i[n]]
        else:
            res = res * np.conj(signal[i[n]])
    return res
    
def get_virtual_location(narray, i):
    res = 0
    for j in range(len(i)//2):
        res = res + narray[i[j]]
    for j in range(len(i)//2, len(i)):
        res = res - narray[i[j]]
    return res

def sort_tuples_by_last_element(tuple_list):
    return sorted(tuple_list, key=lambda x: x[-1])

from collections import defaultdict

def group_tuples_by_last_element(tuple_list):
    groups = defaultdict(list)
    for t in tuple_list:
        groups[t[-1]].append(t[:-1])  # Store all elements except the last
    return dict(groups)

In [3]:
# Set the array parameters (See Thm. II.2 and Eq. 12) 
q = 4
narray = [2]*(2*q+2)
# Set the actual angle
theta = 0.5

# This sets up the simulation that simulates the measured amplitudes at the various physical locations.
# It uses a C=1.5 value, which corresponds to the sampling schedule given in Eq. 16. The variable C here 
# is the parameter K in the paper.
ula_signal = TwoqULASignal(M=narray, C=5)

# Sets up the ESPIRIT object to estimate the amplitude
espirit = ESPIRIT()

signal = ula_signal.estimate_signal(ula_signal.n_samples, theta)

signal

array([ 0.49090909+0.j, -1.        +0.j,  0.15555556+0.j, -0.9       +0.j,
       -0.2       +0.j, -0.06666667+0.j, -0.28      +0.j, -0.9       +0.j,
        0.86666667+0.j, -0.8       +0.j,  0.2       +0.j])

In [4]:
depths = ula_signal.depths
depths

array([  0,   1,   2,   4,   8,  16,  32,  64, 128, 256, 512])

In [5]:
q = 2
N = len(depths)
l=q//2

virtual_locations = []

from itertools import product

comb = product(list(range(N)), repeat=2*q)

I_indices = []
J_indices = []

R = np.zeros((N**q, N**q), dtype=complex)

for i in comb:
    # print(i)
    I_index = index_I(q, N, l, i)
    J_index = index_J(q, N, l, i)
    I_indices.append(I_index)
    J_indices.append(J_index)

    R[I_index][J_index] = cumulant_signal(signal=signal, i=i) #+ R[I_index][J_index] # QUESTION: are the pairs (I_index, J_index) unique for different values of i?
    virtual_locations.append((I_index, J_index, get_virtual_location(narray=depths, i=i)))


In [6]:
index_virtual_location = sort_tuples_by_last_element(virtual_locations)
grouped_tuples = group_tuples_by_last_element(index_virtual_location)

# Print the result
# for key, group in sorted(grouped_tuples.items()):
#     print(f"Group {key}: {group}")

In [7]:
vlocs = np.array(list(grouped_tuples.keys()))
# np.diff(vlocs)

In [8]:
b = np.diff(vlocs)
b = b[:len(b)//2]

start_idx = np.max(np.argwhere(b>1)) + 1
ula = vlocs[start_idx:-start_idx]

ula

array([-72, -71, -70, -69, -68, -67, -66, -65, -64, -63, -62, -61, -60,
       -59, -58, -57, -56, -55, -54, -53, -52, -51, -50, -49, -48, -47,
       -46, -45, -44, -43, -42, -41, -40, -39, -38, -37, -36, -35, -34,
       -33, -32, -31, -30, -29, -28, -27, -26, -25, -24, -23, -22, -21,
       -20, -19, -18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,
        -7,  -6,  -5,  -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,
         6,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,  17,  18,
        19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,
        32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,
        45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,
        58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,
        71,  72])

In [9]:
cvec = np.zeros(len(ula), dtype=complex)

for i, u in enumerate(ula):
    avg_signal = 0
    for idx in grouped_tuples[u]:
        avg_signal += R[idx]
    # avg_signal = avg_signal / len(grouped_tuples[u])
    cvec[i] = avg_signal

# cvec

In [10]:
theta_est, angle = espirit.estimate_theta_toeplitz(cvec[len(cvec)//2:])
# R = ula_signal.get_cov_matrix(cvec)
# theta_est = music.estimate_theta(R, lanczos = False)
# print(angle)
# Estimate the error between estimated a and actual a
theta_est2 = np.sum(np.abs(angle))/8
error = np.abs(np.sin(theta)-np.sin(theta_est2)) 
theta, theta_est, angle, np.sum(np.abs(angle))/8, error

(0.5,
 0.5364142726696303,
 array([-2.14565709,  2.14565709]),
 0.5364142726696303,
 0.03163164494625792)

In [54]:
# Set the array parameters (See Thm. II.2 and Eq. 12) 
q = 4
narray = [2]*(2*q+2)
# Set the actual angle
theta = 0.45532 #theta between 0.0 and 1.57 approx pi/2

# This sets up the simulation that simulates the measured amplitudes at the various physical locations.
# It uses a C=1.5 value, which corresponds to the sampling schedule given in Eq. 16. The variable C here 
# is the parameter K in the paper.
ula_signal = TwoqULASignal(M=narray, C=4)
depths = ula_signal.depths

# Sets up the ESPIRIT object to estimate the amplitude
espirit = ESPIRIT()

signal = ula_signal.estimate_signal(ula_signal.n_samples, theta)

R = ula_signal.get_cov_matrix_toeplitz(signal)
# This estimates the angle using the ESPIRIT algorithm
theta_est, angle = espirit.estimate_theta_toeplitz(R)
# R = ula_signal.get_cov_matrix(signal)
# theta_est = music.estimate_theta(R, lanczos = False)
# print(angle)
# Estimate the error between estimated a and actual a
theta_est2 = np.sum(np.abs(angle))/8
error = np.abs(np.sin(theta)-np.sin(theta_est)) 
theta, theta_est, angle, np.sum(np.abs(angle))/8, error

(0.45532,
 0.6735180386275349,
 array([-2.69407215,  2.69407215]),
 0.6735180386275349,
 0.18398991857431712)

In [9]:
signal

array([ 0.57142857+0.j, -0.91666667+0.j, -0.2       +0.j, -0.375     +0.j,
       -1.        +0.j,  0.25      +0.j, -0.5       +0.j])

In [55]:
def get_vlocs():
    virtual_locations = []
    virtual_signals = []
    difference_matrix = np.zeros((len(depths), len(depths)), dtype=int)
    signal_matrix = np.zeros((len(depths), len(depths)), dtype=complex)
    for r, rval in enumerate(depths):
        for c, cval in enumerate(depths):
            difference_matrix[r][c] = rval-cval
            signal_matrix[r][c] = signal[r] * np.conj(signal[c])
    depths0 = difference_matrix.flatten(order='F')
    signal0 = signal_matrix.flatten(order='F')
    new_depths = depths0
    new_signal = signal0

    virtual_locations.append(depths0)
    virtual_signals.append(signal0)
    for i in range(q-1):
        difference_matrix = np.zeros((len(new_depths), len(depths0)), dtype=int)
        signal_matrix = np.zeros((len(new_depths), len(signal0)), dtype=complex)
        for r, rval in enumerate(new_depths):
            for c, cval in enumerate(depths0):
                difference_matrix[r][c] = rval-cval
                signal_matrix[r][c] = new_signal[r] * np.conj(signal0[c])
        new_depths = difference_matrix.flatten(order='F')
        new_signal = signal_matrix.flatten(order='F')
        virtual_locations.append(new_depths)
        virtual_signals.append(new_signal)

    return virtual_locations, virtual_signals

In [56]:
virtual_locations, virtual_signals = get_vlocs()

In [57]:
virtual_signal = virtual_signals[-1]
virtual_location = virtual_locations[-1]

In [58]:
sorted_idx = np.argsort(virtual_location)

In [59]:
virtual_location[sorted_idx][:100]

array([-2048, -2047, -2047, -2047, -2047, -2046, -2046, -2046, -2046,
       -2046, -2046, -2046, -2046, -2046, -2046, -2045, -2045, -2045,
       -2045, -2045, -2045, -2045, -2045, -2045, -2045, -2045, -2045,
       -2045, -2045, -2045, -2045, -2044, -2044, -2044, -2044, -2044,
       -2044, -2044, -2044, -2044, -2044, -2044, -2044, -2044, -2044,
       -2044, -2044, -2044, -2044, -2044, -2044, -2044, -2044, -2044,
       -2043, -2043, -2043, -2043, -2043, -2043, -2043, -2043, -2043,
       -2043, -2043, -2043, -2043, -2043, -2043, -2043, -2043, -2043,
       -2043, -2043, -2043, -2043, -2043, -2043, -2043, -2043, -2043,
       -2043, -2042, -2042, -2042, -2042, -2042, -2042, -2042, -2042,
       -2042, -2042, -2042, -2042, -2042, -2042, -2042, -2042, -2042,
       -2042])

In [60]:
sorted_vlocs = virtual_location[sorted_idx]

In [61]:
grouped_vlocs = defaultdict(list)
for idx in sorted_idx:
    grouped_vlocs[virtual_location[idx]].append(idx)
# grouped_vlocs

In [62]:
vlocs = np.array(list(grouped_vlocs.keys()))

b = np.diff(vlocs)
b = b[:len(b)//2]

start_idx = np.max(np.argwhere(b>1)) + 1
ula = vlocs[start_idx:-start_idx]

ula

array([-1568, -1567, -1566, ...,  1566,  1567,  1568])

In [77]:
cvec = np.zeros(len(ula), dtype=complex)

for i, u in enumerate(ula):
    avg_signal = 0
    for j, idx in enumerate(grouped_vlocs[u]):
        avg_signal += virtual_signal[idx]
        # if j==4:
        #     break
    # avg_signal = avg_signal / len(grouped_tuples[u])
    cvec[i] = avg_signal

# cvec

In [81]:
theta_est, angle = espirit.estimate_theta_toeplitz(cvec[len(cvec)//2:])
# R = ula_signal.get_cov_matrix(cvec)
# theta_est = music.estimate_theta(R, lanczos = False)
# print(angle)
# Estimate the error between estimated a and actual a
theta_est2 = np.sum(np.abs(angle))/8
error = np.abs(np.sin(theta)-np.sin(theta_est)) 
theta, theta_est, angle, np.sum(np.abs(angle))/8, error

(0.45532,
 1.3184250291803317,
 array([ 1.00948519, -1.00948519]),
 0.2523712976145648,
 0.5285732948911758)